In [ ]:
import pandas as pd
import os
import math
from collections import Counter
import numpy as np
from tqdm import tqdm
tqdm.pandas()

import matplotlib.pyplot as plt

%matplotlib inline

# Gently clean pothole data

In [ ]:
def read_clean(fname, nrows=None, usecols=None, header=0, ftype='csv'):
    if ftype == 'csv':
        df = pd.read_csv('data/raw/'+fname, nrows=nrows, usecols=usecols, header=header)
    elif ftype == 'xlsx':
        df = pd.read_excel('data/raw/'+fname, nrows=nrows, usecols=usecols, header=header)
    cols = df.columns
    new_cols = []
    for c in cols:
        new_cols.append(c.lower().replace(' ', '_').replace(',', ''))
    df.columns = new_cols
    
    return df

In [ ]:
potholes = read_clean('311_Service_Requests_-_Pot_Holes_Reported.csv')
potholes.head()

In [ ]:
potholes.shape

In [ ]:
potholes = potholes[potholes.community_area.notna()]
potholes['community_area'] = potholes.community_area.astype(int)
potholes.shape

In [ ]:
potholes = potholes[(potholes.status == 'Completed') | (potholes.status == 'Completed - Dup')] 
potholes.shape

In [ ]:
potholes.status.value_counts()

In [ ]:
outcomes = {'pothole':['Pothole Patched', 
                       'Complete Upon Arrival',
                       'Completed Upon Arrival',
                       'Completed',       
                       'Follow-on Pot Hole in Street Created',
                       'Create Work Order',
                       'Street Resurfaced',],
            'no_problem':[' No Problem Found',
                          'No Potholes Found',
                          'No Problem Found',],
            'transferred':['CDOT Pavement Cave-In Survey Transfer Outcome',
                           'WM Water Management General Investigation Transfer Outcome',
                           'CDOT Street Cut Complaints Transfer Outcome',
                           'CDOT Pavement Buckle or Speed Hump Transfer Outcome',
                           'GAS Peoples Gas Transfer Outcome',
                           'CDOT Alley Grading - Unimproved Transfer Outcome',
                           'Street Under Construction - Transfer to Inspect Public Way Construction',
                           'WM Sewer Cave In Inspection Transfer Outcome',
                           'WM CDOT Recommended Restoration Transfer Outcome',
                           'CDOT Inspect Public Way Survey Transfer Outcome',
                           'CDOT Alley Pot Hole Transfer Outcome',
                           'CDOT Asphalt Top Off Restoration Transfer Outcome',
                           'CDOT Electrical Operations Construction Complaints Transfer Outcome',
                           'Referred',],
            'bad_request':['No Such Address Found', 
                           'Area Inspected, no such address found',
                           'No Jurisdiction',
                           "Owner's Responsibility",
                           "Private Property - Owner's Responsibility", 
                           'Not Within CDOT Jurisdiction',
                           'No Survey Required',           
                           'No Action - See Remarks in Description',
                           'Survey Completed - No Citation Issued', 
                           'Debris Picked Up'],
           }

def simplify(value, outcomes):
    s = None
    for o in outcomes.keys():
        li = outcomes[o]
        if s is None:
            for l in li:
                if value == l:
                    s = o
                    break
    return s

potholes['outcome'] = potholes.most_recent_action.apply(simplify, outcomes=outcomes)
potholes.head()

In [ ]:
potholes.outcome.value_counts()

In [ ]:
print(potholes.shape)
dates = pd.to_datetime(potholes.creation_date, format="%m/%d/%Y")
start = pd.datetime(year=2011, month=1, day=1)
potholes = potholes[dates >= start]
potholes.shape

In [ ]:
potholes = potholes[['creation_date', 'completion_date', 'service_request_number',
                     'outcome','number_of_potholes_filled_on_block', 'street_address', 
                     #'zip', 'ward', 'police_district',
                     'community_area', 'ssa']]

In [ ]:
def make_blocks(addr):
    parts = addr.upper().split(' ')
    try:
        num = int(parts[0])
        num = math.floor(num/100)*100
        parts[0] = str(num)
        addr = ' '.join(parts)
        
    except:
        print('error with:', addr)
        pass
    
    return addr

potholes['street_address'] = potholes.street_address.progress_apply(make_blocks) #
potholes['street_address'].head()

### traffic data

In [ ]:
df = read_clean('Average_Daily_Traffic_Counts.csv')
df.dtypes

In [ ]:
def clean(sname):
    sname = sname.upper().strip()
    sname = sname.replace('STREET', 'ST')
    return sname

df['street'] = df.street.apply(clean)
df = df.groupby('street')[['total_passing_vehicle_volume']].mean().round(0)#.reset_index()
df.columns = ['traffic_volume']
df['traffic_volume'] = df.traffic_volume.astype(int)
df.head()

In [ ]:
traffic = df.to_dict(orient='index')

In [ ]:
def find_traffic(addr, traffic=traffic):
    t = None
    addr = str(addr)
    for k in traffic.keys():
        if k in addr:
            t = traffic[k]['traffic_volume']
            break
    
    return t

potholes['avg_traffic'] = potholes.street_address.progress_apply(find_traffic)

In [ ]:
potholes.head()

In [ ]:
potholes[['avg_traffic', 'community_area']].describe().round(2)

In [ ]:
potholes.columns = ['creation_date', 'completion_date', 'request_number', 'outcome',
       'n_potholes_on_block', 'address', 'community_area_number', 'ssa', 'avg_traffic']

In [ ]:
potholes.to_csv('data/clean/potholes.tsv', sep='\t', index=False)

# Gather 311 reporting data by area units

In [ ]:
df = read_clean('311_Service_Requests.csv')

In [ ]:
df = df[df.sr_type != '311 INFORMATION ONLY CALL']
#df = df[['sr_number', 'sr_type', 'owner_department', 'created_date', 'street_address',
#         'zip_code', 'duplicate', 'community_area', 'ward',  
#         'police_sector', 'police_district', 'police_beat', 'precinct']]
#df.head()

In [ ]:
grouped_data = dict()
groupings = ['community_area',# 'ward', 'police_sector', 'police_district', 
             #'police_beat', 'zip', 'zip_code', 'precinct', 'ssa'
            ]
for g in groupings:
    tmp = pd.DataFrame()
    tmp[g] = None
    grouped_data[g] = tmp 

In [ ]:
def get_groups(df, existing, cname, idcol='sr_number'):
    for g in existing.keys():
        if g in df.columns:
            tmp = df[[g, idcol]].copy()
            tmp[g] = pd.to_numeric(tmp[g], errors='coerse')
            tmp = tmp.groupby(g).count()
            tmp.columns = [cname]
            tmp.reset_index(inplace=True)
            existing[g] = existing[g].merge(tmp, on=g, how='outer').fillna(0).astype(int)
    return existing

grouped_data = get_groups(df, grouped_data, cname='total_311_calls', idcol='sr_number')

In [ ]:
types = ['Graffiti Removal Request','Street Light Out Complaint', 'Weed Removal Request',
         'Pothole in Street Complaint','Aircraft Noise Complaint',
         'Rodent Baiting/Rat Complaint','Tree Trim Request','Sign Repair Request - All Other Signs',
         'Alley Light Out Complaint','Traffic Signal Out Complaint',
         'Tree Removal Request','Building Violation','Tree Debris Clean-Up Request',
         'Snow - Uncleared Sidewalk/Bike Lane Complaint']

for t in types:
    grouped_data = get_groups(df[df.sr_type == t], grouped_data, cname=t.replace(' ', '_'), idcol='sr_number')
grouped_data['community_area'].head(20)

In [ ]:
for g in grouped_data.keys():
    print(g)
    grouped_data[g].to_csv('data/clean/311_complaints_by_'+g+'.tsv', sep='\t', index=False)

In [ ]:
len(grouped_data['community_area'].community_area.unique())

# SES

In [ ]:
df = read_clean('Census_Data_-_Selected_socioeconomic_indicators_in_Chicago__2008___2012.csv')
df['community_area_number'].fillna(-1, inplace=True)
df['community_area_number'] = df.community_area_number.astype(int)
df.head()

In [ ]:
ah = read_clean('Affordable_Rental_Housing_Developments.csv')

tmp = ah[['community_area_number', 'units']].groupby('community_area_number')
s = tmp.sum()
s.columns = ['n_affordable_housing_units']
s.reset_index(inplace=True)

df = df.merge(s, on='community_area_number', how='outer')
df.head()

In [ ]:
health = read_clean('Public_Health_Statistics-_Life_Expectancy_By_Community_Area.csv')
health = health[['community_area_number', '2010_life_expectancy']]
health['community_area_number'] = health.community_area_number.fillna(-1)
df = df.merge(health, on='community_area_number', how='outer')
df.tail()

In [ ]:
health = read_clean('Public_Health_Statistics-_Selected_public_health_indicators_by_Chicago_community_area.csv')
health['community_area_number'] = health.community_area.fillna(-1).astype(int)
health.dtypes

In [ ]:
health = health[['community_area_number', 'low_birth_weight', 'assault_(homicide)',
                 'cancer_(all_sites)', 'firearm-related', 'infant_mortality_rate', 
                 'childhood_lead_poisoning']]

df = df.merge(health, on='community_area_number', how='outer')
df.tail()

In [ ]:
complaints = read_clean('COPA_Cases_-_Summary.csv')
complaints = complaints[complaints.beat.notnull()]
complaints.tail()

In [ ]:
complaints = complaints.beat.str.split('|').values

beat_list = []
for c in complaints:
    for b in c:
        beat_list.append(int(b.strip()))
        
complaints = pd.DataFrame(Counter(beat_list).most_common(), columns=['beat', 'n_complaints'])
complaints.head()

In [ ]:
cols = ['ID', 'Date', 'Block', 'Primary Type', 'Arrest', 'Beat',
       'District', 'Ward', 'Community Area']
crime = read_clean('Crimes_-_2001_to_present.csv', usecols=cols)
crime.head()

In [ ]:
bca = crime.groupby(['beat', 'community_area'])[['id']].count()
bca = bca.reset_index()
bca.columns = ['beat', 'community_area_number', 'n_bca']

b = crime.groupby(['beat'])[['id']].count()
b.columns = ['n_beat']
b = b.reset_index()

bca = bca.merge(b, on='beat', how='left')
bca['pct_beat_in_ca'] = bca.n_bca/bca.n_beat
bca = bca[['beat', 'community_area_number', 'pct_beat_in_ca']]

bca['community_area_number'] = bca['community_area_number'].astype(int)

bca.head()

In [ ]:
complaints = complaints.merge(bca, on='beat', how='left')
complaints = complaints.dropna(subset=['community_area_number'])
complaints.head()

In [ ]:
complaints = complaints[complaints.community_area_number > 0]
tmp = pd.DataFrame([{'community_area_number': -1, 
                     'n_complaints': complaints.n_complaints.sum(), 
                     'pct_beat_in_ca':1}])
tmp
complaints = pd.concat([complaints, tmp])
complaints.tail()

In [ ]:
complaints['police_complaints'] = complaints.n_complaints * complaints.pct_beat_in_ca
complaints = complaints.groupby('community_area_number')[['police_complaints']].sum().reset_index()
complaints.head(10)

In [ ]:
df = df.merge(complaints, on='community_area_number', how='outer')
df.tail()

In [ ]:
cols = ['geogkey', 'total_population',
       'not_hispanic_or_latino_white_alone',
       'median_age', 
       'total_housing_units', 
       'vacant_housing_units', 
       'renter_occupied']

demog = read_clean('CCASF12010CMAP.xlsx', ftype='xlsx', header=1)
demog=demog[cols]
demog.head()

In [ ]:
demog.dtypes

In [ ]:
demog['pct_white'] = demog.not_hispanic_or_latino_white_alone / demog.total_population * 100
demog['pct_vacant_housing'] = demog.vacant_housing_units / demog.total_housing_units * 100
demog['pct_rental_housing'] = demog.renter_occupied / demog.total_housing_units * 100
demog.head()

In [ ]:
demog = demog[['geogkey', 'total_population','median_age','pct_white', 
              'pct_vacant_housing', 'pct_rental_housing', 'total_housing_units']]
demog.columns = ['community_area_number', 'total_population','median_age','pct_white', 
              'pct_vacant_housing', 'pct_rental_housing', 'total_housing_units']
demog.tail()

In [ ]:
df = df.merge(demog, on='community_area_number', how='outer')
df['pct_affordable_housing'] = df.n_affordable_housing_units / df.total_housing_units * 100
df['police_complaints_per_thousand'] = df.police_complaints / df.total_population * 100
df.head()

In [ ]:
df = df[['community_area_number', 'community_area_name',
       'percent_of_housing_crowded', 'percent_households_below_poverty',
       'percent_aged_16+_unemployed',
       'percent_aged_25+_without_high_school_diploma','per_capita_income_',
       'hardship_index', '2010_life_expectancy',
       'low_birth_weight', 'assault_(homicide)', 'cancer_(all_sites)',
       'firearm-related', 'infant_mortality_rate', 'childhood_lead_poisoning',
       'police_complaints_per_thousand', 'total_population', 'median_age', 'pct_white',
       'pct_vacant_housing', 'pct_rental_housing', 'total_housing_units',
       'pct_affordable_housing']]

In [ ]:
df.to_csv('data/clean/community_area_stats.tsv', sep='\t', index=False)